In [1]:
from theano import gpuarray
%matplotlib inline
from __future__ import print_function, division
path = "data/lesson3_HW_Kaggle/"
#path = "data/lesson3_HW_Kaggle/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 73, in init_dev
    pygpu_version.fullversion)
ValueError: Your installed version of pygpu(0.6.9) is too old, please upgrade to 0.7.0 or later (but below 0.8.0)
Using Theano backend.


## Create Train/Valid datasets

In [2]:
import make_train_val as mtv
import os
import numpy as np

In [3]:
pwd

u'/home/ubuntu/courses/deeplearning1/nbs'

In [4]:
path = os.getcwd() + '/data/lesson3_HW_Kaggle/'
path

'/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/'

In [5]:
#G Get all files in all directories following this pattern
from glob import glob
g = glob(path + 'train/'+ 'c?/*.jpg')
g[:5]
len(g) * .80

17939.2

In [6]:
shuf = np.random.permutation(g)
shuf

array(['/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/train/c5/img_86839.jpg',
       '/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/train/c6/img_80954.jpg',
       '/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/train/c0/img_18149.jpg', ...,
       '/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/train/c6/img_36131.jpg',
       '/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/train/c6/img_15442.jpg',
       '/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/train/c2/img_14852.jpg'],
      dtype='|S85')

In [7]:
p = '/'.join(path.split('/')[1:-1][-2:])
p

'data/lesson3_HW_Kaggle'

In [8]:
path.split('/')[1:-1][-2:]

['data', 'lesson3_HW_Kaggle']

In [9]:
path

'/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/'

In [10]:
classes = ['c'+str(i) for i in range(10)]
classes

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [11]:
mtv.create_sample_dirs(path, classes)
mtv.create_train_val_dirs(path, classes)

Samples directories created with copies of files for train and validation
Validation files moved from training directory.
Validation size: 0.2


## Load pretrained weights from ResNet

In [9]:
!tree -d './data/lesson3_HW_Kaggle/'

./data/lesson3_HW_Kaggle/
├── sample
│   ├── train
│   │   ├── c0
│   │   ├── c1
│   │   ├── c2
│   │   ├── c3
│   │   ├── c4
│   │   ├── c5
│   │   ├── c6
│   │   ├── c7
│   │   ├── c8
│   │   └── c9
│   └── valid
│       ├── c0
│       ├── c1
│       ├── c2
│       ├── c3
│       ├── c4
│       ├── c5
│       ├── c6
│       ├── c7
│       ├── c8
│       └── c9
├── test
├── train
│   ├── c0
│   ├── c1
│   ├── c2
│   ├── c3
│   ├── c4
│   ├── c5
│   ├── c6
│   ├── c7
│   ├── c8
│   └── c9
└── valid
    ├── c0
    ├── c1
    ├── c2
    ├── c3
    ├── c4
    ├── c5
    ├── c6
    ├── c7
    ├── c8
    └── c9

46 directories


In [14]:
path = os.getcwd() + '/data/lesson3_HW_Kaggle/'
path

'/home/ubuntu/courses/deeplearning1/nbs/data/lesson3_HW_Kaggle/'

In [15]:
# create a batch iterator for training and validation image sets
train_bathces = get_batches(path+'train/', batch_size=64)
val_batches = get_batches(path+'valid/', batch_size=64, shuffle=False)

Found 17939 images belonging to 10 classes.
Found 4485 images belonging to 10 classes.


In [20]:
# get the one-hot encoded labels and other info 
val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames = get_classes(path)

#something here causing test set not to batch (look into sample being in the same level directory)

Found 17939 images belonging to 10 classes.
Found 4485 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


In [22]:
# Load the VGG CNN architecture
vgg = Vgg16()
model = vgg.model